In [66]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import dash
from dash import html, dcc, Input, Output

In [67]:
standard_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Standard_radar_chart_v2.csv")

In [68]:
attacking_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Attacking_radar_chart_v2.csv")

In [69]:
defending_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Defending_radar_chart_v2.csv")

In [29]:
import plotly.graph_objects as go

In [30]:
team_data = standard_chart_data[standard_chart_data['TEAM_NAME'] == 'Brentford']

In [31]:
import numpy as np

In [75]:
average_data = standard_chart_data[standard_chart_data['TEAM_NAME'] == 'AVERAGE']
average_data[average_data['SEASON'] == 2223]

,SEASON,TEAM_NAME,Stats,Values,Norm_Values
56,2223,AVERAGE,Pass Completion (%),78.570378,0.490848
57,2223,AVERAGE,Tackles Won (%),58.335138,0.530159
58,2223,AVERAGE,Goals/Game,1.426316,0.368254
59,2223,AVERAGE,NpxG/Game,1.321000,0.428125
60,2223,AVERAGE,Conceded/Game,1.426316,0.528889
61,2223,AVERAGE,xG Against/Game,1.421500,0.378804
62,2223,AVERAGE,Shots/Game,12.513158,0.471888
63,2223,AVERAGE,Shots On Target (%),33.085983,0.553671


In [77]:
def create_radar_chart(season, team_name, data, chart_name):
    team_data = data[data['TEAM_NAME'] == team_name]
    average_data = data[data['TEAM_NAME'] == 'AVERAGE']

    team_data = team_data[team_data['SEASON'] == season]
    average_data = average_data[average_data['SEASON'] == season]

    # Prepare data for plotting
    categories = team_data['Stats']
    norm_values = team_data['Norm_Values']
    average_norm_values = average_data['Norm_Values']
    values = team_data['Values']

    difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

    hover_text = (
    "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
    "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
    "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
    )

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=average_norm_values,
        theta=categories,
        name='AVERAGE',
        fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
        line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_trace(go.Scatterpolar(
        r=norm_values,
        theta=categories,
        name=team_name,
        opacity=0.6,
        fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
        line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_layout_image(
        dict(
            source='https://i.imgur.com/9yKFcv4.png',
            xref="paper", yref="paper",
            x=0.16, y=1,
            sizex=1.05, sizey=1.05,
            opacity=0.7,  # Adjust opacity as needed
            layer="below"
        )
    )

    for i, (value, category) in enumerate(zip(values, categories)):
        angle = (i / float(len(categories))) * 2 * np.pi 
        x = 0.5 + (1.1) * np.cos(angle) / 2
        y = 0.48 + (1.1) * np.sin(angle) / 1.8

        annotation_text = \
        f"<span style='font-size: 15px;'><b>{category}</b></span><br>" \
        f"<span style='line-height: 2px;'>&nbsp;</span><br>" \
        f"<span style='font-size: 20px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

        fig.add_annotation(
            x=x,
            y=y,
            xref="paper",
            yref="paper",
            text=annotation_text,  # Bold category name and value
            showarrow=False,
            font=dict(size=10, color='white'),
            align="center",
            xanchor='center',
            yanchor='middle',
        )

    # Update layout
    fig.update_layout(
        autosize=False,
        width=800,  # Set the width
        height=600,  # Set the height
        polar=dict(
            bgcolor='rgba(0,0,0,0)',
            radialaxis=dict(
                visible=False,
                range=[0, 1],
                linecolor='rgba(17, 17, 17, 1)',
                showline=False,
                gridcolor='white'
            ),
            angularaxis=dict(
                showline=False,  # Hide angular axis line
                gridcolor='rgba(0,0,0,0)',
                showticklabels=False  
            )
        ),
        paper_bgcolor='rgb(70, 70, 70)',      
        showlegend=False,
        title=f'{chart_name} for {team_name}',
        hoverlabel=dict(
            bgcolor="rgba(20, 20, 20, 0.8)",
            font_family="Roboto, sans-serif",
            bordercolor="rgba(20, 20, 20, 0.8)",),
        font=dict(
            family="Roboto, sans-serif",  # Specify the font family
            size=15,                     # Specify the font size
            color="white"                # Specify the font color
        )
    )

    return fig



In [82]:
# def standard_radar_chart(team_name, data):
#     # Filter data for the selected team
#     team_data = data[data['TEAM_NAME'] == team_name]
#     average_data = data[data['TEAM_NAME'] == 'AVERAGE']

#     # Prepare data for plotting
#     categories = team_data['Stats']
#     norm_values = team_data['Norm_Values']
#     average_norm_values = average_data['Norm_Values']
#     values = team_data['Values']
    

#     difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

#     hover_text = (
#     "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
#     "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
#     "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
#     )

#     # Create radar chart
#     fig = go.Figure()

#     fig.add_trace(go.Scatterpolar(
#         r=average_norm_values,
#         theta=categories,
#         name='AVERAGE',
#         fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
#         line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
#         fill='toself',
#         customdata=np.stack((values, difference), axis=-1),
#         hovertemplate=hover_text,
#         marker=dict(
#             size=1  # Hides the markers by setting their size to zero
#         )
#     ))

#     fig.add_trace(go.Scatterpolar(
#         r=norm_values,
#         theta=categories,
#         name=team_name,
#         opacity=0.6,
#         fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
#         line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
#         fill='toself',
#         customdata=np.stack((values, difference), axis=-1),
#         hovertemplate=hover_text,
#         marker=dict(
#             size=1  # Hides the markers by setting their size to zero
#         )
#     ))

#     fig.add_layout_image(
#         dict(
#             source='https://i.imgur.com/9yKFcv4.png',
#             xref="paper", yref="paper",
#             x=0.16, y=1,
#             sizex=1.05, sizey=1.05,
#             opacity=0.7,  # Adjust opacity as needed
#             layer="below"
#         )
#     )

#     for i, (value, category) in enumerate(zip(values, categories)):
#         angle = (i / float(len(categories))) * 2 * np.pi 
#         x = 0.5 + (1.1) * np.cos(angle) / 2
#         y = 0.48 + (1.1) * np.sin(angle) / 1.8

#         annotation_text = \
#         f"<span style='font-size: 15px;'><b>{category}</b></span><br>" \
#         f"<span style='line-height: 2px;'>&nbsp;</span><br>" \
#         f"<span style='font-size: 20px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

#         fig.add_annotation(
#             x=x,
#             y=y,
#             xref="paper",
#             yref="paper",
#             text=annotation_text,  # Bold category name and value
#             showarrow=False,
#             font=dict(size=10, color='white'),
#             align="center",
#             xanchor='center',
#             yanchor='middle',
#         )

#     # Update layout
#     fig.update_layout(
#         autosize=False,
#         width=800,  # Set the width
#         height=600,  # Set the height
#         polar=dict(
#             bgcolor='rgba(0,0,0,0)',
#             radialaxis=dict(
#                 visible=False,
#                 range=[0, 1],
#                 linecolor='rgba(17, 17, 17, 1)',
#                 showline=False,
#                 gridcolor='white'
#             ),
#             angularaxis=dict(
#                 showline=False,  # Hide angular axis line
#                 gridcolor='rgba(0,0,0,0)',
#                 showticklabels=False  
#             )
#         ),
#         paper_bgcolor='rgb(70, 70, 70)',      
#         showlegend=False,
#         title=f'Radar Chart for {team_name}',
#         hoverlabel=dict(
#             bgcolor="rgba(20, 20, 20, 0.8)",
#             font_family="Roboto, sans-serif",
#             bordercolor="rgba(20, 20, 20, 0.8)",),
#         font=dict(
#             family="Roboto, sans-serif",  # Specify the font family
#             size=15,                     # Specify the font size
#             color="white"                # Specify the font color
#         )
#     )

#     return fig

# Create and show the radar chart for a specific team
fig = create_radar_chart(2223, 'Fulham', defending_chart_data, "Defending Radar Chart")
fig.show()

In [ ]:
def standard_radar_chart(team_name, data):
    # Filter data for the selected team
    team_data = data[data['TEAM_NAME'] == team_name]
    average_data = data[data['TEAM_NAME'] == 'AVERAGE']

    # Prepare data for plotting
    categories = team_data['Stats']
    norm_values = team_data['Norm_Values']
    average_norm_values = average_data['Norm_Values']
    values = team_data['Values']
    

    difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

    hover_text = (
    "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
    "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
    "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
    )

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=average_norm_values,
        theta=categories,
        name='AVERAGE',
        fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
        line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_trace(go.Scatterpolar(
        r=norm_values,
        theta=categories,
        name=team_name,
        opacity=0.6,
        fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
        line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_layout_image(
        dict(
            source='https://i.imgur.com/9yKFcv4.png',
            xref="paper", yref="paper",
            x=0.16, y=1,
            sizex=1.05, sizey=1.05,
            opacity=0.7,  # Adjust opacity as needed
            layer="below"
        )
    )

    for i, (value, category) in enumerate(zip(values, categories)):
    # Calculate the angle for the category
        angle = (i / float(len(categories))) * 2 * np.pi 
        x = 0.5 + (1.1) * np.cos(angle) / 2
        y = 0.48 + (1.1) * np.sin(angle) / 1.8

        annotation_text = \
        f"<span style='font-size: 15px;'><b>{category}</b></span><br>" \
        f"<span style='line-height: 2px;'>&nbsp;</span><br>" \
        f"<span style='font-size: 20px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

        fig.add_annotation(
            x=x,
            y=y,
            xref="paper",
            yref="paper",
            text=annotation_text,  # Bold category name and value
            showarrow=False,
            font=dict(size=10, color='white'),
            align="center",
            # The xanchor and yanchor are set to 'middle' and 'center' to center the annotations
            xanchor='center',
            yanchor='middle',
        )

    # Update layout
    fig.update_layout(
        autosize=False,
        width=800,  # Set the width
        height=600,  # Set the height
        polar=dict(
            bgcolor='rgba(0,0,0,0)',
            radialaxis=dict(
                visible=False,
                range=[0, 1],
                linecolor='rgba(17, 17, 17, 1)',
                showline=False,
                gridcolor='white'
            ),
            angularaxis=dict(
                showline=False,  # Hide angular axis line
                gridcolor='rgba(0,0,0,0)',
                showticklabels=False  
            )
        ),
        paper_bgcolor='rgb(70, 70, 70)',      
        showlegend=False,
        title=f'Radar Chart for {team_name}',
        hoverlabel=dict(
            bgcolor="rgba(20, 20, 20, 0.8)",
            font_family="Roboto, sans-serif",
            bordercolor="rgba(20, 20, 20, 0.8)",),
        font=dict(
            family="Roboto, sans-serif",  # Specify the font family
            size=15,                     # Specify the font size
            color="white"                # Specify the font color
        )
    )

    return fig

# Create and show the radar chart for a specific team
fig = standard_radar_chart('Manchester City', standard_chart_data)
fig.show()